<a href="https://colab.research.google.com/github/rjsgk5792/darknet/blob/master/n122a_chi_square_test_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- `민간부문`만 포함합니다
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from google.colab import files

uploaded = files.upload()

Saving 규모별_미분양현황_20220126143907.csv to 규모별_미분양현황_20220126143907 (1).csv


In [21]:
import io
import pandas as pd

df = pd.read_csv('규모별_미분양현황_20220126143907.csv',encoding='cp949')
df.drop('부문(1)',axis=1,inplace=True)
df

,시도(1),규모(1),2020. 08
0,서울,60㎡이하,54
1,서울,60∼85㎡,2
2,서울,85㎡초과,0
3,부산,60㎡이하,577
4,부산,60∼85㎡,735
5,부산,85㎡초과,142
6,대구,60㎡이하,143
7,대구,60∼85㎡,1437
8,대구,85㎡초과,44
9,대전,60㎡이하,782


In [28]:
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '대전', '대구', '부산']
value = list(df.iloc[:,2])
df = df.pivot(index='시도(1)',columns='규모(1)',values='2020. 08')
df
# 인덱스 순서 어떻게 바꿀까?
# df.columns = col
# df = df.reindex(row)

규모(1),60∼85㎡,60㎡이하,85㎡초과
시도(1),,,
대구,1437,143,44
대전,1,782,0
부산,735,577,142
서울,2,54,0


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [52+2+0, 590+665+142, 113+1061+42, 772+1+0]
```
을 비교 하게 될 것입니다.




In [45]:
from scipy.stats import chisquare


df_sum = df.sum(axis=1)
print(df_sum)
chi1 = chisquare(df_sum,axis=None)
chi1
# 지역별 미분양 수에 차이가 있다.

시도(1)
대구    1624
대전     783
부산    1454
서울      56
dtype: int64


Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [51]:
from scipy.stats import chi2_contingency

print(chi2_contingency(df))
print('---'*30)
print(chi2_contingency(df,correction = False))

# 가설
# H0 : 지역별 미분양가구의 수와 규모의 상관 관계(차이)가 있다.

# p-value 가 0.0 이므로 H0 기각.



(2064.5767314171994, 0.0, 6, array([[901.76155221, 645.12228746,  77.11616033],
       [434.77789124, 311.04110288,  37.18100587],
       [807.36533061, 577.59101353,  69.04365586],
       [ 31.09522594,  22.24559612,   2.65917794]]))
------------------------------------------------------------------------------------------
(2064.5767314171994, 0.0, 6, array([[901.76155221, 645.12228746,  77.11616033],
       [434.77789124, 311.04110288,  37.18100587],
       [807.36533061, 577.59101353,  69.04365586],
       [ 31.09522594,  22.24559612,   2.65917794]]))


### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

In [59]:
# 카이제곱 검정 : (실제값-예측값)^2/예측값   들의 합
import numpy as np

obs = df_sum
exp = np.mean(obs)

squared = np.power(obs-exp,2)
chi = np.sum(squared/exp)
chi

1564.4572376818994

## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [ ]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
    return None# 내용을 채우세요.

myChisq(v1) 

# chisquare값과 pvalue를 출력해야합니다.

myChisq(v2)

### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [ ]:
# 이 곳에 답안을 작성하시길 바랍니다